In [ ]:
from minisom import MiniSom
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon
from matplotlib import cm

from bokeh.plotting import figure
from bokeh.io import save, show, output_file, output_notebook

output_notebook()

In [ ]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt', 
                   names=['area', 'perimeter', 'compactness', 'length_kernel', 'width_kernel',
                          'asymmetry_coefficient', 'length_kernel_groove', 'target'], 
                   sep='\t+')

data = data[data.columns[:-1]]
# data normalization
data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
data = data.values

# Initialization and training
som = MiniSom(15, 15, data.shape[1], sigma=1.5, learning_rate=.7, activation_distance='euclidean',
              topology='hexagonal', neighborhood_function='gaussian', random_seed=10)

som.train(data, 1000, verbose=True)

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111)

ax.set_aspect('equal')

xx, yy = som.get_euclidean_coordinates()
umatrix = som.distance_map()
weights = som.get_weights()

for i in range(weights.shape[0]):
    for j in range(weights.shape[1]):
        wy = yy[(i, j)] * 2 / np.sqrt(3) * 3 / 4
        hex = RegularPolygon((xx[(i, j)], wy), 
                             numVertices=6, 
                             radius=.95 / np.sqrt(3),
                             facecolor=cm.Blues(umatrix[i, j]), 
                             alpha=.4, 
                             edgecolor='gray')
        ax.add_patch(hex)
for x in data:
    w = som.winner(x) 
    # place a marker on the winning position for the sample xx
    wx, wy = som.convert_map_to_euclidean(w) 
    wy = wy * 2 / np.sqrt(3) * 3 / 4
    plt.plot(wx, wy, 
             markerfacecolor='None',
             markeredgecolor='black', 
             markersize=12, 
             markeredgewidth=2)

plt.show()
plt.savefig('outputs/example_stackoverflow_matplotlib.png')

***

## HexagonalTopology: bokeh

In [ ]:
from bokeh.colors import RGB
from bokeh.models import LogColorMapper
# convert matplotlib colour palette to bokeh hex
cm_blues = (255 * cm.Blues(range(256))).astype('int')
palette_blues = [RGB(*tuple(rgb)).to_hex() for rgb in cm_blues]
color_mapper = LogColorMapper(palette=palette_blues, low=1, high=1e7)

In [ ]:
tile_centres_column = []
tile_centres_row = []
colours = []
for i in range(weights.shape[0]):
    for j in range(weights.shape[1]):
        wy = yy[(i, j)] * 2 / np.sqrt(3) * 3 / 4
        tile_centres_column.append(xx[(i, j)])
        tile_centres_row.append(wy)
        colours.append(cm.Blues(umatrix[i, j]))
        #colours.append(palette_blues)
        
weight_x = []
weight_y = []
for x in data:
    w = som.winner(x)
    wx, wy = som.convert_map_to_euclidean(xy=w)
    wy = wy * 2 / np.sqrt(3) * 3/4
    weight_x.append(wx)
    weight_y.append(wy)

In [ ]:
plot = figure(plot_width=800, plot_height=800,
              #x_range=(0, weights.shape[0]), y_range=(0, weights.shape[1]),
              match_aspect=True) 
plot.hex_tile(q=tile_centres_column, r=tile_centres_row, 
              size=.95 / np.sqrt(3),
              color=colours,
              fill_alpha=.4,
              line_color='black')
plot.dot(x=weight_x, y=weight_y,
         fill_color='black',
         size=12)

show(plot)
output_file("outputs/example_stackoverflow_bokeh.html")
save(plot)

In [ ]:
from bokeh.models import ColumnDataSource, Plot, Hex, Dot
from bokeh.transform import linear_cmap
from bokeh.io import curdoc

plot = Plot(title=None, plot_width=800, plot_height=800, match_aspect=True)

source_hex = ColumnDataSource(dict(
    x=tile_centres_column,
    y=tile_centres_row,
    c=colours
))

source_dot = ColumnDataSource(dict(
    wx=weight_x,
    wy=weight_y
))

hex = Hex(y='x', x='y',
          size=100*(.95 / np.sqrt(3)),
          fill_color=linear_cmap('counts', 'Viridis256', 0, ))
plot.add_glyph(source_or_glyph=source_hex, glyph=hex)

dot = Dot(y='wx', x='wy', size=30, line_color='black')
plot.add_glyph(source_or_glyph=source_dot, glyph=dot)

curdoc().add_root(plot)
show(plot)